## Intermediate Cleaning - Simply Recipes

In [1]:
# read in kickstarter intermediate data 
import numpy as np
import os
import pandas as pd 
import seaborn as sns
import sys
import re
# import warnings
# warnings.filterwarnings('ignore')

# src_dir = os.path.join(os.getcwd(), '..', '..', 'src')
# sys.path.append(src_dir)

# from d01_data.utils import read_multiple_csv_and_concat

In [2]:
recipes_sr1 = pd.read_csv('../../data/01_raw/simply_recipes/simply_recipes_1.csv')
recipes_sr2 = pd.read_csv('../../data/01_raw/simply_recipes/simply_recipes_2.csv')
recipes_sr3 = pd.read_csv('../../data/01_raw/simply_recipes/simply_recipes_3.csv')

## Drop Useless Columns & Concat

Looks like both Datasets have unnamed columns. Let's drop these

In [3]:
recipes_sr1.columns

Index(['Unnamed: 0', 'title', 'prep_time', 'cook_time', 'recipe_yield', 'tags',
       'ingredients', 'entire_card', 'byline', 'link_food'],
      dtype='object')

In [4]:
recipes_sr2.columns

Index(['Unnamed: 0', 'title', 'prep_time', 'cook_time', 'recipe_yield', 'tags',
       'ingredients', 'entire_card', 'byline', 'link_food'],
      dtype='object')

In [5]:
recipes_sr3.columns

Index(['Unnamed: 0', 'title', 'prep_time', 'cook_time', 'recipe_yield', 'tags',
       'ingredients', 'entire_card', 'byline', 'link_food'],
      dtype='object')

In [6]:
recipes_sr1.drop(columns=['Unnamed: 0'], inplace=True)
recipes_sr2.drop(columns=['Unnamed: 0'], inplace=True)
recipes_sr3.drop(columns=['Unnamed: 0'], inplace=True)

In [7]:
print(recipes_sr1.columns)
print(recipes_sr2.columns)
print(recipes_sr3.columns)

Index(['title', 'prep_time', 'cook_time', 'recipe_yield', 'tags',
       'ingredients', 'entire_card', 'byline', 'link_food'],
      dtype='object')
Index(['title', 'prep_time', 'cook_time', 'recipe_yield', 'tags',
       'ingredients', 'entire_card', 'byline', 'link_food'],
      dtype='object')
Index(['title', 'prep_time', 'cook_time', 'recipe_yield', 'tags',
       'ingredients', 'entire_card', 'byline', 'link_food'],
      dtype='object')


### Concat

In [8]:
recipes_sr_full = pd.concat([recipes_sr1, recipes_sr2, recipes_sr3])
recipes_sr_full.reset_index(inplace=True, drop=True)

In [9]:
recipes_sr_full.head()

,title,prep_time,cook_time,recipe_yield,tags,ingredients,entire_card,byline,link_food
0,['Grilled Cheese BLT'],"['Prep time:', ' ', '10 minutes']","['Cook time:', ' ', '10 minutes']","['Yield:', ' ', '4 sandwiches']","['Filed under:', ' ', 'Dinner', 'Lunch', 'Sand...","['\n ', 'Ingredients', ...","['\n\n ', '\n ...","['by ', ' ', 'Aaron Hutcherson', 'August 2...","['<link rel=""canonical"" href=""https://www.simp..."
1,['Pulled Pork Sandwich'],"['Prep time:', ' ', '10 minutes']","['Cook time:', ' ', '2 hours, 45 minutes']","['Yield:', ' ', 'Serves 6 to 8']","['Filed under:', ' ', 'Dinner', 'Sandwich', 'B...","['\n ', 'Ingredients', ...","['\n\n ', '\n ...","['by ', ' ', 'Elise Bauer', 'Updated Augus...","['<link rel=""canonical"" href=""https://www.simp..."
2,['How to Make Bacon in the Oven'],"['Prep time:', ' ', '5 minutes']","['Cook time:', ' ', '20 minutes']","['Yield:', ' ', '12 strips']","['Filed under:', ' ', 'Tips', 'Breakfast and B...","['\n ', 'Ingredients', ...","['\n\n ', '\n ...","['by ', ' ', 'Nick Evans', 'August 25, 2019']","['<link rel=""canonical"" href=""https://www.simp..."
3,['Sausage Stuffed Zucchini'],"['Prep time:', ' ', '15 minutes']","['Cook time:', ' ', '1 hour']","['Yield:', ' ', 'Serves 4']","['Filed under:', ' ', 'Dinner', 'Favorite Summ...","['\n ', 'Ingredients', ...","['\n\n ', '\n ...","['by ', ' ', 'Elise Bauer', 'Updated Augus...","['<link rel=""canonical"" href=""https://www.simp..."
4,['The Best Dry Rub for Ribs'],"['Prep time:', ' ', '5 minutes']","['Yield:', ' ', '1 1/2 cups']",[],"['Filed under:', ' ', 'Favorite Fall', 'Favori...","['\n ', 'Ingredients', ...","['\n\n ', '\n ...","['by ', ' ', 'Irvin Lin', 'July 28, 2019']","['<link rel=""canonical"" href=""https://www.simp..."


## EDA - Let's see what we have

**TO-DO LIST**
1. title - take title out of brackets
1. prep_time - take cook time down to time
1. cook_time - split into 2 columns (one with number and other with measure word) 
1. tags - make tags just a simple list
1. ingredients - make simple list


In [10]:
recipes_sr_full.drop_duplicates(inplace=True)

In [11]:
recipes_sr_full.shape

(1748, 9)

In [12]:
recipes_sr_full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1748 entries, 0 to 1751
Data columns (total 9 columns):
title           1748 non-null object
prep_time       1748 non-null object
cook_time       1748 non-null object
recipe_yield    1748 non-null object
tags            1748 non-null object
ingredients     1748 non-null object
entire_card     1748 non-null object
byline          1748 non-null object
link_food       1748 non-null object
dtypes: object(9)
memory usage: 136.6+ KB


### Title

In [13]:
titles = recipes_sr_full.title

In [14]:
title_new = []
for title in titles:
    try:
        title_new.append(re.search("'(.*)\'", title).group(1))
    except:
        title_new.append(np.nan)

### Prep Time

In [15]:
prep_time = recipes_sr_full.prep_time

In [16]:
prep_time_new = []
for time in prep_time:
    try:
        prep_time_new.append(re.search("\'Prep time:\', \' \',\ \'(.*)\'", time).group(1))
    except:
        prep_time_new.append(np.nan)

### cook_time

In [17]:
cook_times = recipes_sr_full.cook_time

In [18]:
cook_time_new = []
for time in cook_times:
    try:
        cook_time_new.append(re.search("\'Cook time:\', \' \',\ \'(.*)\'", time).group(1))
    except:
        cook_time_new.append(np.nan)

### recipe_yield

In [19]:
recipe_yield = recipes_sr_full.recipe_yield

# clean the recipe yield
recipe_yield_new = []
for element in recipe_yield:
    try:
        recipe_yield_new.append(re.search("\'Yield:\', \' \', \'(.*)\']", element).group(1))
    except:
        recipe_yield_new.append(np.nan)

### Tags

In [20]:
tags = recipes_sr_full.tags

In [21]:
tags_new = []
for tag in tags:
    sub_list = []
    try:
        sub_list.append(re.search("\'Filed under:\', \' \',\ (.*)\]", tag).group(1))
    except:
        sub_list.append(np.nan)
    tags_new.append(sub_list)

### Ingredients

In [22]:
ingredient_lists = recipes_sr_full.ingredients

In [23]:
ingredient_lists[0]

"['\\n                        ', 'Ingredients', '\\n\\n                        ', '\\n', '8 slices sourdough bread', '\\n', '4 tablespoon unsalted butter, at room temperature', '\\n', '8 ounces (2 cups) shredded cheddar cheese', '\\n', '2 slicing tomatoes (such as beefsteak, Brandywine, or Cherokee purple), sliced 1/4-inch thick', '\\n', '8 to 12 slices ', 'cooked bacon', '\\n', '12 leaves butterhead or other crispy lettuce', '\\n', 'Kosher salt and black pepper', '\\n', '\\n\\n\\t\\t\\t\\t\\t\\t\\n                                              ']"

In [24]:
count = 0
while count <= 3:
    count = count + 1
    for sub_list in ingredient_lists:
        for idx, element in enumerate(sub_list):
            try:
                if element == "Ingredients":
                    sub_list.pop(idx)
                if element == "'":
                    sub_list.pop(idx)
                if element == "":
                    sub_list.pop(idx)
                if element == "                        ', '":
                    sub_list.pop(idx)
                if element == "                        ', 'For the sauce:', '":
                    sub_list.pop(idx)
                if element == "                                                  ', ' Special equipment:', '":
                    sub_list.pop(idx)
                if '\\\\t' in element:
                    sub_list.pop(idx)
                if element == "                                              '":
                    sub_list.pop(idx)
                if element == '':
                    sub_list.pop(idx)
            except:
                pass

In [25]:
ingredients_new = []
for ingredient_list in ingredient_lists:
    try:
        ingredients_new.append(re.findall("\'(.*?)\'", ingredient_list))
    except:
        ingredients_new.append(np.nan)

In [26]:
ingredients_new[1]

['\\n                        ',
 'Ingredients',
 '\\n\\n                        ',
 'For the sauce:',
 '\\n',
 '\\n',
 '1 large onion, chopped',
 '\\n',
 '6 garlic cloves, peeled',
 '\\n',
 '1 pickled jalapeño pepper, seeded and chopped',
 '\\n',
 '2 teaspoons Chipotle chile powder',
 '\\n',
 '1 tablespoon tomato paste',
 '\\n',
 '2 tablespoon Dijon mustard',
 '\\n',
 '3/4 cup distilled white vinegar',
 '\\n',
 '1 teaspoon paprika',
 '\\n',
 '1/3 cup ketchup',
 '\\n',
 '2 teaspoons Worcestershire sauce',
 '\\n',
 '1/4 cup light brown sugar',
 '\\n',
 'Salt',
 '\\n',
 '\\n',
 'For the pork:',
 '\\n',
 '\\n',
 '1 bay leaf',
 '\\n',
 '3 pounds pork butt shoulder roast',
 '\\n',
 'Hamburger buns',
 '\\n',
 '\\n\\n\\t\\t\\t\\t\\t\\t\\n                                              ']

In [27]:
count = 0
while count <= 3:
    count = count + 1
    for sub_list in ingredients_new:
        for idx, element in enumerate(sub_list):
            try:
                if element == "Ingredients":
                    sub_list.pop(idx)
                if element == "'":
                    sub_list.pop(idx)
                if element == "":
                    sub_list.pop(idx)
                if element == "                        ', '":
                    sub_list.pop(idx)
                if element == "                        ', 'For the sauce:', '":
                    sub_list.pop(idx)
                if element == "                                                  ', ' Special equipment:', '":
                    sub_list.pop(idx)
                if '\\\\t' in element:
                    sub_list.pop(idx)
                if element == "                                              '":
                    sub_list.pop(idx)
                if element == '':
                    sub_list.pop(idx)
                if "                        ', '" in element:
                    sub_list.pop(idx)
#                 if element == "', '":
#                     sub_list.pop(idx)
            except:
                pass

In [28]:
ingredients_new

[['\\n                        ',
  '\\n\\n                        ',
  '\\n',
  '8 slices sourdough bread',
  '\\n',
  '4 tablespoon unsalted butter, at room temperature',
  '\\n',
  '8 ounces (2 cups) shredded cheddar cheese',
  '\\n',
  '2 slicing tomatoes (such as beefsteak, Brandywine, or Cherokee purple), sliced 1/4-inch thick',
  '\\n',
  '8 to 12 slices ',
  'cooked bacon',
  '\\n',
  '12 leaves butterhead or other crispy lettuce',
  '\\n',
  'Kosher salt and black pepper',
  '\\n',
  '\\n\\n\\t\\t\\t\\t\\t\\t\\n                                              '],
 ['\\n                        ',
  '\\n\\n                        ',
  'For the sauce:',
  '\\n',
  '\\n',
  '1 large onion, chopped',
  '\\n',
  '6 garlic cloves, peeled',
  '\\n',
  '1 pickled jalapeño pepper, seeded and chopped',
  '\\n',
  '2 teaspoons Chipotle chile powder',
  '\\n',
  '1 tablespoon tomato paste',
  '\\n',
  '2 tablespoon Dijon mustard',
  '\\n',
  '3/4 cup distilled white vinegar',
  '\\n',
  '1 tea

In [29]:
# Note, this line needs to be run three times. Fix this later.
count = 0
while count <= 3:
    for sub_list in ingredients_new:
        for idx, element in enumerate(sub_list):
            if "\\n" in element:
                sub_list.pop(idx)
    count = count + 1

In [30]:
ingredients_new[0]

['8 slices sourdough bread',
 '4 tablespoon unsalted butter, at room temperature',
 '8 ounces (2 cups) shredded cheddar cheese',
 '2 slicing tomatoes (such as beefsteak, Brandywine, or Cherokee purple), sliced 1/4-inch thick',
 '8 to 12 slices ',
 'cooked bacon',
 '12 leaves butterhead or other crispy lettuce',
 'Kosher salt and black pepper']

In [31]:
nl = []
for i in ingredients_new:
    sl = []
    for ele in i:
        sl.append(ele.strip('.][\,'))
    nl.append(sl)

In [32]:
nl[0]

['8 slices sourdough bread',
 '4 tablespoon unsalted butter, at room temperature',
 '8 ounces (2 cups) shredded cheddar cheese',
 '2 slicing tomatoes (such as beefsteak, Brandywine, or Cherokee purple), sliced 1/4-inch thick',
 '8 to 12 slices ',
 'cooked bacon',
 '12 leaves butterhead or other crispy lettuce',
 'Kosher salt and black pepper']

In [33]:
data_sr = []
for i in nl:
    sl = []
    for ele in i:
        sl.append(ele.strip("').,('"))
    data_sr.append(sl)

In [34]:
data_sr[70]

['1 tablespoons extra virgin olive oil',
 '1 yellow onion,\\xa0finely chopped (about 1\\xa01/2 to 2 cups',
 '2 cloves garlic, minced (about 2 teaspoons',
 '4 ounces (110 g) Italian sweet sausage (or one 7 to 8 inch link',
 '4 ounces (110 g) \\xa0Italian hot and spicy sausage\\xa0(or one 7 to 8 inch link',
 '1\\xa0large (28 oz, 794 g) can of whole peeled tomatoes',
 '1 pound (16 oz, 450 g) spaghetti pasta',
 'Salt',
 'Freshly grated Parmesan cheese']

### byline'

In [35]:
bylines = list(recipes_sr_full.byline)

In [36]:
len(bylines)

1748

In [37]:
byline_new = []
for byline in bylines:
    try:
        byline_new.append(re.search("\[\'by   \', \'   \', \'(.*)\', \'", byline).group(1))
    except: 
        byline_new.append(byline)

In [38]:
len(byline_new)

1748

In [39]:
byline_new_2 = []
for byline in byline_new:
    try:
        byline_new_2.append(re.search("\[\'by   \', \'   \', \'(.*)\'\]", byline).group(1))
    except: 
        byline_new_2.append(byline)

In [40]:
len(byline_new_2)

1748

In [41]:
byline_new_3 = []
for byline in byline_new_2:
    try:
        byline_new_3.append(re.search("\[\'   \', \'   \', (.*)\'\]", byline).group(1))
    except: 
        byline_new_3.append(byline)

In [42]:
len(byline_new_3)

1748

### link_food

In [43]:
link_food = list(recipes_sr_full.link_food)

In [44]:
link_food_new = []
for link in link_food:
    try:
        link_food_new.append(re.search("\\\'<link rel=\"canonical\" href=\"(.*)\"\>\\\'\]", link).group(1))
    except: 
        link_food_new.append(link)

In [45]:
print('Title Count: ',len(title_new))
print('Prep Time Count: ', len(prep_time_new))
print('Cook Time Count: ',len(cook_time_new))
print('Tag Count',len(tags_new))
print('Ingredients Count: ',len(data_sr))
print('Recipe Yield Count: ',len(recipe_yield_new))
print('Byline Count: ',len(byline_new_3))
print('Link Count: ', len(link_food_new))

Title Count:  1748
Prep Time Count:  1748
Cook Time Count:  1748
Tag Count 1748
Ingredients Count:  1748
Recipe Yield Count:  1748
Byline Count:  1748
Link Count:  1748


In [46]:
recipe_sr_full_final = pd.DataFrame()

In [47]:
recipe_sr_full_final['title'] = title_new
recipe_sr_full_final['prep_time'] = prep_time_new
recipe_sr_full_final['cook_time'] = cook_time_new
recipe_sr_full_final['tags'] = tags_new
recipe_sr_full_final['ingredients'] = data_sr
recipe_sr_full_final['recipe_yield'] = recipe_yield_new
recipe_sr_full_final['byline'] = byline_new_3
recipe_sr_full_final['link_food'] = link_food_new

In [49]:
recipe_sr_full_final

,title,prep_time,cook_time,tags,ingredients,recipe_yield,byline,link_food
0,Grilled Cheese BLT,10 minutes,10 minutes,"['Dinner', 'Lunch', 'Sandwich', 'Favorite Summ...","[8 slices sourdough bread, 4 tablespoon unsalt...",4 sandwiches,Aaron Hutcherson,https://www.simplyrecipes.com/recipes/grilled_...
1,Pulled Pork Sandwich,10 minutes,"2 hours, 45 minutes","['Dinner', 'Sandwich', 'Budget', 'Comfort Food...","[For the sauce:, 1 large onion, chopped, 6 gar...",Serves 6 to 8,Elise Bauer,https://www.simplyrecipes.com/recipes/pulled_p...
2,How to Make Bacon in the Oven,5 minutes,20 minutes,"['Tips', 'Breakfast and Brunch', 'Baking', 'Ho...","[12 strips bacon, 1/2 teaspoon ground black pe...",12 strips,Nick Evans,https://www.simplyrecipes.com/recipes/how_to_m...
3,Sausage Stuffed Zucchini,15 minutes,1 hour,"['Dinner', 'Favorite Summer', 'Make-ahead', 'I...","[2 tablespoons extra virgin olive oil, 1/2 pou...",Serves 4,Elise Bauer,https://www.simplyrecipes.com/recipes/italian_...
4,The Best Dry Rub for Ribs,5 minutes,NaN,"['Favorite Fall', 'Favorite Summer', 'Game Day...",[3/4 cup packed dark brown sugar (or 1/2 cup i...,NaN,Irvin Lin,https://www.simplyrecipes.com/recipes/the_best...
5,Ginger Pork Rice Bowls,15 minutes,20 minutes,"['Eat Your Food', 'Family Dinner Ideas', 'Dinn...","[For the bowls:, 1 tablespoon olive oil, 1/2 c...",4-6 servings,Nick Evans,https://www.simplyrecipes.com/recipes/ginger_p...
6,Hawaiian SPAM Tacos with Pineapple,10 minutes,30 minutes,"['Eat Your Food', 'Dinner', 'Kid-friendly', 'H...","[DAD ADD: Haba, ñero Guacamole, 1 habañero pep...","12 small tacos, serving 4 adults",Nick Evans,https://www.simplyrecipes.com/recipes/hawaiian...
7,Air Fryer Chinese Egg Rolls,20 minutes,25 minutes,"['Appetizer', 'Snack', 'Air Fryer', 'Quick and...","[For the egg rolls:, 1 tablespoon olive oil, 1...",12 egg rolls,Nick Evans,https://www.simplyrecipes.com/recipes/air_frye...
8,Grilled Bacon-Wrapped Stuffed Hot Dogs,10 minutes,11 minutes,"['Dinner', 'Favorite Summer', 'Grill', 'Bacon'...","[1 teaspoon ketchup, 1 teaspoon Dijon mustard,...",Makes 4 stuffed hot dogs,Elise Bauer,https://www.simplyrecipes.com/recipes/grilled_...
9,Italian Grilled Cheese Sandwiches,10 minutes,20 minutes,"['Eat Your Food', 'Dinner', 'Sandwich', 'Kid-f...","[For the DAD ADD: , Olive and Cauliflower Giar...",4 sandwiches,Nick Evans,https://www.simplyrecipes.com/recipes/italian_...


In [ ]:
# recipe_sr_full_final.to_csv('../../data/02_intermediate/recipies_sr_inter.csv')

In [ ]:
# recipe_sr_full_final.to_pickle("../../data/02_intermediate/recipies_sr_inter.pkl")

## LET'S MAKE A FUNCTION

In [ ]:
# def intermediate_clean_recipes_sr(unclean_concated_dataset)
#     # let's clean the title
#     titles = recipes_sr_full.title
#     title_new = []
#     for title in titles:
#         try:
#             title_new.append(re.search("'(.*)\'", title).group(1))
#         except:
#             title_new.append(np.nan)

#     # clean prep time column 
#     prep_time = recipes_sr_full.prep_time
#     prep_time_new = []
#     for time in prep_time:
#         try:
#             prep_time_new.append(re.search("\'Prep time:\', \' \',\ \'(.*)\'", time).group(1))
#         except:
#             prep_time_new.append(np.nan)

#     # clean cook time column 
#     cook_times = recipes_sr_full.cook_time
#     cook_time_new = []
#     for time in cook_times:
#         try:
#             cook_time_new.append(re.search("\'Cook time:\', \' \',\ \'(.*)\'", time).group(1))
#         except:
#             cook_time_new.append(np.nan)

#     # clean the recipe yield column   
#     recipe_yield = recipes_sr_full.recipe_yield
#     recipe_yield_new = []
#     for element in recipe_yield:
#         try:
#             recipe_yield_new.append(re.search("\'Yield:\', \' \', \'(.*)\']", element).group(1))
#         except:
#             recipe_yield_new.append(np.nan)

#     # clean tags column 
#     tags = recipes_sr_full.tags
#     tags_new = []
#     for tag in tags:
#         sub_list = []
#         try:
#             sub_list.append(re.search("\'Filed under:\', \' \',\ (.*)\]", tag).group(1))
#         except:
#             sub_list.append(np.nan)
#         tags_new.append(sub_list)

#     # clean ingredients column 
#     ingredient_lists = recipes_sr_full.ingredients
#     count = 0
#     while count <= 3:
#         count = count + 1
#         for sub_list in ingredient_list:
#             for idx, element in enumerate(sub_list):
#                 try:
#                     if element == "Ingredients":
#                         sub_list.pop(idx)
#                     if element == "'":
#                         sub_list.pop(idx)
#                     if element == "":
#                         sub_list.pop(idx)
#                     if element == "                        ', '":
#                         sub_list.pop(idx)
#                     if element == "                        ', 'For the sauce:', '":
#                         sub_list.pop(idx)
#                     if element == "                                                  ', ' Special equipment:', '":
#                         sub_list.pop(idx)
#                     if '\\\\t' in element:
#                         sub_list.pop(idx)
#                     if element == "                                              '":
#                         sub_list.pop(idx)
#                     if element == '':
#                         sub_list.pop(idx)
#                 except:
#                     pass

#     # take out new line characters 
#     count = 0
#     while count <= 3:
#         for sub_list in ingredients_new:
#             for idx, element in enumerate(sub_list):
#                 if "\\n" in element:
#                     sub_list.pop(idx)
#         count = count + 1

#     # strip punctuation 
#     nl = []
#     for i in ingredients_new:
#         sl = []
#         for ele in i:
#             sl.append(ele.strip('.][\,'))
#         nl.append(sl)

#     # strip punctuation 2
#     data_sr = []
#     for i in nl:
#         sl = []
#         for ele in i:
#             sl.append(ele.strip("').,('"))
#         data_sr.append(sl)

#     # clean byline
#     bylines = list(recipes_sr_full.byline)
#     byline_new = []
#     for byline in bylines:
#         try:
#             byline_new.append(re.search("\[\'by   \', \'   \', \'(.*)\', \'", byline).group(1))
#         except: 
#             byline_new.append(byline)

#     byline_new_2 = []
#     for byline in byline_new:
#         try:
#             byline_new_2.append(re.search("\[\'by   \', \'   \', \'(.*)\'\]", byline).group(1))
#         except: 
#             byline_new_2.append(byline)

#     byline_new_3 = []
#     for byline in byline_new_2:
#         try:
#             byline_new_3.append(re.search("\[\'   \', \'   \', (.*)\'\]", byline).group(1))
#         except: 
#             byline_new_3.append(byline)

#     # clean link 
#     link_food = list(recipes_sr_full.link_food)
#     link_food_new = []
#     for link in link_food:
#         try:
#             link_food_new.append(re.search("\\\'<link rel=\"canonical\" href=\"(.*)\"\>\\\'\]", link).group(1))
#         except: 
#             link_food_new.append(link)

#     # make empty dataset
#     recipe_sr_full_final = pd.DataFrame()
#     # make empty thing. 
#     recipe_sr_full_final['title'] = title_new
#     recipe_sr_full_final['prep_time'] = prep_time_new
#     recipe_sr_full_final['cook_time'] = cook_time_new
#     recipe_sr_full_final['tags'] = tags_new
#     recipe_sr_full_final['ingredients'] = data_sr
#     recipe_sr_full_final['recipe_yield'] = recipe_yield_new
#     recipe_sr_full_final['byline'] = byline_new_3
#     recipe_sr_full_final['link_food'] = link_food_new
    
#     return recipe_sr_full_final